### Packages

In [2]:
from pandas_datareader import data as pdr
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import cvxpy as cvx
from scipy.stats import norm
from numpy.linalg import inv
import math
import benchmarks

# for plotting
import plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
import matplotlib.pyplot as plt

# Initialize plotly offline
plotly.offline.init_notebook_mode(connected=True)

In [7]:
import warnings
warnings.filterwarnings('ignore')

### Data functions

In [8]:
# Download Stock data from Yahoo Finance
def download_stock_data(start_date, end_date, asset_list, filename):
    """
    asset_list: list of asset to download
    filename: name of the csv file where the data go to
    """
    for i, asset in enumerate(asset_list):
        if i == 0:
            data = pdr.get_data_yahoo(asset_list[i], start_date,end_date)[['Adj Close']]
            all_prices = data.rename(columns={"Adj Close": asset})
            
        else:
            new_asset = pdr.get_data_yahoo(asset_list[i], start_date,end_date)[['Adj Close']]
            all_prices = all_prices.join(new_asset.rename(columns={"Adj Close": asset}))
    
    all_prices.to_csv('./00_Data/' + filename + '.csv')
    return

# Extract Return and accumulated price data (starting value 1) from raw stock prices
def process_stock_data(all_prices):
    """
    input:
    all_prices: data frame, price of a list of stock from start date to the end date
    
    return:
    all_price_norm: normalized asset price with the price on the first day as 1
    all_returns: relative price
    """
    n_time, n_assets = all_prices.shape
    # Calculate returns
    all_returns = all_prices.pct_change(1).apply(lambda a : a + 1)
    
    # Calculate relative price time series
    all_prices_norm = all_returns.copy()
    for i in range(n_time):
        if i == 0:
            all_prices_norm.iloc[0] = np.ones(n_assets)
        else:
            for j in range(n_assets):
                if math.isnan(all_prices_norm.iloc[i,j]):
                    all_prices_norm.iloc[i,j] = all_prices_norm.iloc[i-1,j]
                else:
                    all_prices_norm.iloc[i,j] = all_prices_norm.iloc[i,j] * all_prices_norm.iloc[i-1,j]
                    
    return all_prices_norm, all_returns.dropna()

#1/fluc, 1, 1/fluc, 1, ...
def teststock1(T, fluc):
    stock = np.ones(T+1)
    for i in range(T+1):
        if i%2 == 1:
            stock[i] = 1.0/fluc
    return stock

#Brownian Motion
def teststock2(T, delta, sigma):
    stock1 = np.zeros(T+1)
    stock1[0] = 3.0
    for i in range(T):
        stock1[i+1] = stock1[i] + sigma + norm.rvs(scale=delta**2)
        if (stock1[i+1] <= 0):
            print('Negative Warning!')
    return stock1

#Garbage 1, 1/fluc, 1/pow(fluc,2) ,...
def teststock3(T, discount):
    stock = np.ones(T+1, dtype=float)
    for i in range(T):
        stock[i+1] = stock[i]*discount
    return stock

In [9]:
# # download stock data and safe as csv in /Data folder
# commodities = ['GC=F','SI=F', 'CL=F'] #gold, silver, Crude Oil
# bonds = ['^FVX'] #5 Year US Government Bonds
# us_stocks = ['SPY', # The SP500 as a benchmark
#              'AAPL', 'MMM', 'BA' , 'CAT', 'CVX' , 'CSCO', 'KO' , 'DIS',
#              'GS', 'IBM', 'INTC', 'JNJ', 'JPM', 'MSFT', 'PFE'] #Some stocks from the Dow Jones

# start_data = '2019-01-01'
# end_date = '2019-7-11'

# #Download and save data as csv
# #download_stock_data(start_data, end_date, commodities, 'Commodities_latest')
# #download_stock_data(start_data, end_date, bonds, 'bonds_latest')
# # download_stock_data(start_data, end_date, us_stocks, 'us_stocks_latest')

KeyboardInterrupt: 

In [10]:
# read in the data again
us_stock_prices = pd.read_csv('./00_Data/us_stocks_latest.csv', header=0, index_col = 0)
us_stock_prices.index = pd.to_datetime(us_stock_prices.index)

#Process Stock Data
stock_prices_norm, stock_returns = process_stock_data(us_stock_prices)

# Extract benchmark SP500 Data
SPY_benchmark = stock_prices_norm.drop(us_stocks[1:], axis=1)
stock_prices_norm_wo_SPY = stock_prices_norm.drop(us_stocks[0], axis=1)

# filter out benchmark data from stock returns
stock_returns = stock_returns.drop(us_stocks[0], axis=1)

In [11]:
# Plot prices
iplot(us_stock_prices.iplot(asFigure=True, kind='scatter',xTitle='Dates',yTitle='Prices',title='Prices'))

# Plot returns
iplot(stock_returns.iplot(asFigure=True, kind='scatter',xTitle='Dates',yTitle='Returns',title='Returns'))

# Plot normalized prices
iplot(stock_prices_norm.iplot(asFigure=True, kind='scatter',xTitle='Dates',yTitle='Relative Price',title='Relative Prices'))

In [12]:
#Scraper

url = 'https://finance.yahoo.com/most-active?count=100&offset=0'
r = requests.get(url)
data = r.text
soup = BeautifulSoup(data)

stock_names = [link.string for link in soup.find_all('a', class_='Fw(600)')]
len(stock_names)

7

In [13]:
# start_date, end_date = '2001-01-01', '2019-7-18'

# download_stock_data(start_date, end_date, stock_names, 'hundred_stocks')

KeyboardInterrupt: 

In [ ]:
# # read in the data again
# stock_prices = pd.read_csv('./00_Data/hundred_stocks.csv', header=0, index_col = 0)
# stock_prices.index = pd.to_datetime(stock_prices.index)

# #Process Stock Data
# stock_prices_norm, stock_returns = process_stock_data(stock_prices)

In [ ]:
# # Plot prices
# iplot(stock_prices.iplot(asFigure=True, kind='scatter',xTitle='Dates',yTitle='Prices',title='Prices'))

# # Plot returns
# iplot(stock_returns.iplot(asFigure=True, kind='scatter',xTitle='Dates',yTitle='Returns',title='Returns'))

# # Plot normalized prices
# iplot(stock_prices_norm.iplot(asFigure=True, kind='scatter',xTitle='Dates',yTitle='Relative Price',title='Relative Prices'))

### Minimum spanning tree

We are to find the minimum spanning tree using [Kruskal's algorithm](https://en.wikipedia.org/wiki/Kruskal%27s_algorithm).

Implementation and tutorial [here](https://www.geeksforgeeks.org/kruskals-minimum-spanning-tree-algorithm-greedy-algo-2/).

In [ ]:
# Helper functions

def historical_mean(historical_returns):
    return historical_returns.sum(axis = 0)

def distance(i, j):
    return np.sqrt(2 * (1 - correlation_matrix.iloc[i, j]))

In [ ]:
# Get stock prices of selected stocks during subprime mortgage crisis
subprime_mortgage_crisis = stock_prices.loc['2008-09'].dropna(axis=1) 
SMPC_returns = process_stock_data(subprime_mortgage_crisis)[1] # H_i,t of each asset i for t = 1,...,20

# Construct correlation matrix with zeroes
correlation_matrix = pd.DataFrame(np.zeros((SMPC_returns.shape[1], SMPC_returns.shape[1])))

In [ ]:
# Name the columns and rows in DataFrame
stock_names = SMPC_returns.columns
num_stocks = len(stock_names)
correlation_matrix.index, correlation_matrix.columns = stock_names, stock_names

In [ ]:
# Fill in the correlation matrix
for i in range(num_stocks):
    for j in range(num_stocks):
        top_left = historical_mean(SMPC_returns.iloc[:, i] * SMPC_returns.iloc[:, j])
        top_right = historical_mean(SMPC_returns.iloc[:, i]) * historical_mean(SMPC_returns.iloc[:, j])
        numerator = top_left - top_right
        
        bottom_left = historical_mean(SMPC_returns.iloc[:, i] ** 2) - historical_mean(SMPC_returns.iloc[:, i]) ** 2
        bottom_right = historical_mean(SMPC_returns.iloc[:, j] ** 2) - historical_mean(SMPC_returns.iloc[:, j]) ** 2
        denominator = np.sqrt(bottom_left * bottom_right)
        
        correlation_matrix.iloc[i, j] = numerator / denominator

In [ ]:
# Construct euclidean distances matrix with zeroes
euclidean_distance_matrix = pd.DataFrame(np.zeros((SMPC_returns.shape[1], SMPC_returns.shape[1])))
euclidean_distance_matrix.index, euclidean_distance_matrix.columns = stock_names, stock_names
# Fill in the euclidean distances matrix
for i in range(num_stocks):
    for j in range(num_stocks):
        euclidean_distance_matrix.iloc[i, j] = distance(i, j)

In [ ]:
from collections import defaultdict
#Class to represent a graph
class Graph:
    def __init__(self, vertices):
        self.V = vertices  #No. of vertices
        self.graph = []  # default dictionary
        # to store graph

    # function to add an edge to graph
    def addEdge(self, u, v, w):
        self.graph.append([u, v, w])

    # A utility function to find set of an element i
    # (uses path compression technique)
    def find(self, parent, i):
        if parent[i] == i:
            return i
        return self.find(parent, parent[i])

    # A function that does union of two sets of x and y
    # (uses union by rank)
    def union(self, parent, rank, x, y):
        xroot = self.find(parent, x)
        yroot = self.find(parent, y)

        # Attach smaller rank tree under root of
        # high rank tree (Union by Rank)
        if rank[xroot] < rank[yroot]:
            parent[xroot] = yroot
        elif rank[xroot] > rank[yroot]:
            parent[yroot] = xroot

        # If ranks are same, then make one as root
        # and increment its rank by one
        else:
            parent[yroot] = xroot
            rank[xroot] += 1

    # The main function to construct MST using Kruskal's
    # algorithm
    def KruskalMST(self):

        result = []  #This will store the resultant MST

        i = 0  # An index variable, used for sorted edges
        e = 0  # An index variable, used for result[]

        # Step 1:  Sort all the edges in non-decreasing
        # order of their
        # weight.  If we are not allowed to change the
        # given graph, we can create a copy of graph
        self.graph = sorted(self.graph, key=lambda item: item[2])

        parent = []
        rank = []

        # Create V subsets with single elements
        for node in range(self.V):
            parent.append(node)
            rank.append(0)

        # Number of edges to be taken is equal to V-1
        while e < self.V - 1:

            # Step 2: Pick the smallest edge and increment
            # the index for next iteration
            u, v, w = self.graph[i]
            i = i + 1
            x = self.find(parent, u)
            y = self.find(parent, v)

            # If including this edge does't cause cycle,
            # include it in result and increment the index
            # of result for next edge
            if x != y:
                e = e + 1
                result.append([u, v, w])
                self.union(parent, rank, x, y)
            # Else discard the edge

        # print the contents of result[] to display the built MST
#         print("Following are the edges in the constructed MST")
#         for u, v, weight in result:
#             print(str(u) + " -- " + str(v) + " == " + str(weight))
        return result

In [ ]:
# Set up tree
g = Graph(num_stocks)
for i in range(num_stocks):
    for j in range(num_stocks):
        g.addEdge(i, j, distance(i, j))

In [ ]:
# Get minimum spanning tree
MST = g.KruskalMST()
MST = pd.DataFrame(MST)
col1, col2 = list(MST.iloc[: , 0]), list(MST.iloc[: , 1])

In [ ]:
# Get leaves
leaves = []
for i in range(num_stocks):
    if col1.count(i) == 1 ^ col2.count(i) == 1:
        leaves.append(i)

In [ ]:
leaves

In [ ]:
leave_stocks = [stock_names[i] for i in leaves]

In [ ]:
leave_stocks

### Method 1: Risk-Aware Multi-Armed Bandit Problem

We are first to implement the risk-aware multi-armed bandit method. This includes two parts: $\omega _t^M$, the single-asset multi-armed bandit portfolio at $t$, and $\omega _t^C$, the risk-aware portfolio.

#### 1. Single-asset multi-armed bandit portfolio

In [18]:
def Bandit_Portfolio(stock_returns):
    n_time, n_assets = stock_returns.shape # n_time >> n_assets in our case
    portfolio = np.zeros((n_assets, n_time)) # the output portfolio
    Rbar = np.zeros((n_assets,1)) # empirical mean of return for assets

    num_selected = {}
    for i in range(n_assets):
        num_selected[i] = 0
        
    # for loop up until n_time, output tye bandit portfolio at each time t
    
    for t in range(n_time):
        
        if t < n_assets:
            portfolio[t, t] = 1
            Rbar[t] = stock_returns.iloc[t, t]
            num_selected[t] = 1
            continue
            
        max_asset = 0
        max_upper_bound = 0
        for asset in range(n_assets):
            
            avg_reward = Rbar[asset]
            right_part = np.sqrt(2*np.log(t) / num_selected[asset])
            upper_bound = avg_reward + right_part
            
            if upper_bound > max_upper_bound:
                max_asset = asset
                max_upper_bound = upper_bound
        portfolio[max_asset, t] = 1
        #pull
        Rbar[max_asset] = (Rbar[max_asset] * num_selected[max_asset] + stock_returns.iloc[t, max_asset]) / (num_selected[max_asset] + 1)
        num_selected[max_asset] += 1
        
    return portfolio

In [23]:
print(np.sum(Bandit_Portfolio(stock_returns), axis = 1, keepdims = True))

[[1.]
 [1.]
 [1.]
 [2.]
 [1.]
 [1.]
 [2.]
 [2.]
 [1.]
 [1.]
 [1.]
 [2.]
 [2.]
 [2.]
 [1.]]


#### 2. Risk-aware portfolio

In [ ]:
def Risk_Aware_Portfolio(stock_returns):
    